# Analysis of APT data (statistical distributions)

**System**
* Alloy composition: $Al_{0.1}FeNiCrCo$
* Lattice: FCC
* Lattice constant: 3.57A

**APT measurement**
* Detector efficiency: 33%
* position uncertainty (standard deviation): 5A (in x-, y-directions) and 3A in z-direction

## Setup

In [1]:
# Notebook settings

# code development settings (automatically reload modified modules)
%load_ext autoreload
%autoreload 2

# plotting backend
%matplotlib inline

In [2]:
# basic libraries

import os, sys
import re
import copy
import glob
from itertools import product
import pickle
from collections import Counter
import numpy as np
#import scipy.io
from scipy.stats import multinomial, chi2
from scipy.optimize import fmin
from sklearn.neighbors import NearestNeighbors

# plotting
import matplotlib.pyplot as plt

# data handling
# import pyUSID as usid
# import h5py
# import pandas as pd
# import xarray as xr

# parallelization
# from subprocess import Popen, PIPE
import multiprocessing as mp

In [134]:
# local library
sys.path.append('../../../force_fields/statmechlib')
from statmechlib.read_write import read_lattice_mc, read_modeldef, write_modeldef
from statmechlib.preprocessing import get_stats_latt, cfg_replicate, latt_to_real_coords, add_experimental_noise
from statmechlib.preprocessing import real_coords, select_core, get_knn_stats
from statmechlib.forcefields import sd2, get_chi2_two, get_s2_two, data_lists, loss_sd2_hist
from statmechlib.forcefields import make_reference_matrices, make_target_matrices

In [289]:
# project directories
data_raw = '../data/target_raw'
src_dir = '../src/utils'
sim_dir = '../sim/enth_model'
sim_dir_m1 = '../sim/m1'
sim_dir_m2 = '../sim/m2'
sim_dir_m3 = '../sim/m3'

In [5]:
sys.path.append(src_dir)
from utils import *

## Read target data

In [6]:
# compositions, concentrations, atom types
element = ['Al','Fe','Ni','Cr','Co','X']

ntype = len(element)

# numbers of KNNs of a particular atom i of type ti
knn = list(range(1,13))
knn += [20, 40, 100, 200, 500, 1000]

# directories containing data for different KNNs
knn_dirs = [os.path.join(data_raw, 'K'+str(k)) for k in knn]

print(knn_dirs)

['../data/target_raw/K1', '../data/target_raw/K2', '../data/target_raw/K3', '../data/target_raw/K4', '../data/target_raw/K5', '../data/target_raw/K6', '../data/target_raw/K7', '../data/target_raw/K8', '../data/target_raw/K9', '../data/target_raw/K10', '../data/target_raw/K11', '../data/target_raw/K12', '../data/target_raw/K20', '../data/target_raw/K40', '../data/target_raw/K100', '../data/target_raw/K200', '../data/target_raw/K500', '../data/target_raw/K1000']


In [7]:
targets = {}
for d, k in zip(knn_dirs, knn):
    targets[k] = read_pairwise(os.path.join(d, f'input312_PatternK{k}_PairWise'), k, ntype)

In [8]:
# count statistics and concentrations
counts = np.sum(targets[knn[5]], axis=2)[:,0]
assert np.array_equal(counts, np.sum(targets[knn[0]], axis=2)[:,0])

conc = counts/np.sum(counts)
tot_counts = np.sum(counts)

In [370]:
conc, counts

(array([0.02325016, 0.22784748, 0.22782936, 0.21274711, 0.21713616,
        0.09118974]),
 array([ 30788, 301717, 301693, 281721, 287533, 120754]),
 6)

In [383]:
targets[1]

array([[[ 30089,    699],
        [ 23806,   6982],
        [ 23474,   7314],
        [ 24353,   6435],
        [ 24113,   6675],
        [ 28105,   2683]],

       [[294673,   7044],
        [232809,  68908],
        [232738,  68979],
        [236829,  64888],
        [236242,  65475],
        [275294,  26423]],

       [[294379,   7314],
        [232910,  68783],
        [233035,  68658],
        [237308,  64385],
        [235961,  65732],
        [274872,  26821]],

       [[275243,   6478],
        [217007,  64714],
        [217680,  64041],
        [221591,  60130],
        [220129,  61592],
        [256955,  24766]],

       [[280785,   6748],
        [221556,  65977],
        [221539,  65994],
        [225780,  61753],
        [225862,  61671],
        [262143,  25390]],

       [[117961,   2793],
        [ 93640,  27114],
        [ 93644,  27110],
        [ 95702,  25052],
        [ 94901,  25853],
        [107922,  12832]]])

In [385]:
kksum = 0
for kk in targets[1][:]:
    ks = sum([i*p for i, p in enumerate(kk[-1])])
    print(ks)
    kksum += ks
print(kksum)

2683
26423
26821
24766
25390
12832
118915


In [384]:
counts

array([ 30788, 301717, 301693, 281721, 287533, 120754])

## Generate random distributions (null hypothesis)

* Assume that there are no interactioins between particles and for each KNN generate multinomial distributions with parameters (probabilities) set to mole fractions.

In [9]:
null_reference = {}
null_counts = {}
for k in knn:
    null_reference[k] = get_binomials(k, conc)
    null_counts[k] = null_reference[k]*tot_counts

## Read reference data from MC simulations

In [290]:
# read in a set of trajectories
trj_dirs = ['txinf', 'tx500', 'tx700', 'tx1000']
#trj_dirs = ['1x700']
trj_dirs = ['3x700']

In [291]:
#trjs = {}
for key in trj_dirs:
    # read trajectory info
    trjs[key] = read_lattice_mc(os.path.join(sim_dir_m3, key), verbose=1)

Reading ../sim/m3/3x700/lg.hst
Reading ../sim/m3/3x700/lg.run
Reading ../sim/m3/3x700/lg.xyz
Reading ../sim/m3/3x700/lg.mld


In [292]:
trjs['3x700']['ref_params']

array([ 0.  , -2.97, -7.  , -1.05, -4.  ,  0.  , -0.78, -0.06, -0.48,
        0.  , -0.24, -0.17,  0.  ,  0.04,  0.  ])

## Add experimental noise

In [293]:
unknown_frac = counts[-1]/np.sum(counts)

# analyze configurations
for key, trj in trjs.items():
    if key not in ['3x700']:
        continue
    print('trj:', key)

    # convert from lattice to real coordinates
    trj = latt_to_real_coords(trj, 3.57/2.0)

    # model of experimental noise
    xyzs, typs = add_experimental_noise(trj, loss_rate=0.67, disp=np.array([5.0, 5.0, 3.0]), unknown_frac=unknown_frac)
    trjs[key]['xyz_noise'] = xyzs
    trjs[key]['atom_type_noise'] = typs

trj: 3x700


In [334]:
#[(k, v[0:5]) for k, v in trjs['1x700'].items()]
unknown_frac

0.09118973936079432

In [295]:
# 1. replicate configurations to avoid dealing with periodic boundary conditions
# 2. convert to real coordinates
# 3. select core
for key, trj in trjs.items():
    if key not in ['3x700']:
        continue

    print('trj:', key)
    xyzs = []
    boxs = []
    tis = []
    for xyz, box, ti in zip(trj['xyz_noise'], trj['box'], trj['atom_type_noise']):
        xyz_big, box_big = cfg_replicate(xyz, box, vec_a=2, vec_b=2, vec_c=2)
        xyzs.append(xyz_big)
        boxs.append(box_big)
        tis.append(np.tile(ti, 2*2*2))                                          

    trj['xyz_noise'] = xyzs
    trj['box_noise'] = boxs
    trj['atom_type_noise'] = tis
    
    trj['xyz_noise'] = real_coords(trj['xyz_noise'], trj['box_noise'])
    
    trj['xyz_core'], trj['atom_type_core'] = select_core(trj['xyz_noise'], trj['box_noise'], trj['atom_type_noise'])

trj: 3x700


In [296]:
# with open('../data/working/trjs.pickle', 'wb') as fo:
#     pickle.dump(trjs, fo)

### Neighbor statistics

In [297]:
knns = list(range(1,13))
knns += [20, 40, 100, 200]

# create nighbor model with the maximum of nearest neighbors (1000). KDTree should make this Nlog(N)
neigh = NearestNeighbors(n_neighbors=knns[-1], algorithm='kd_tree', n_jobs=4)

In [298]:
for key, trj in trjs.items():
    if key not in ['3x700']:
        continue

    print('trj:', key)

    knn_stats = []
    for it, (cor, xyz, ti_core, ti) in enumerate(zip(trj['xyz_core'], trj['xyz_noise'], trj['atom_type_core'], trj['atom_type_noise'])):
        print(it, len(ti_core), len(ti))
        if it < 2000:
            continue
        neigh.fit(xyz)
        dist, ind = neigh.kneighbors(cor, n_neighbors=knns[-1]+1)
        assert np.array_equal(dist[:,0], np.zeros(dist.shape[0])), "First neighbor is not center"
        knn_stats.append(get_knn_stats(ind, ti_core, ti, knns=knns))

    trj['knn'] = knn_stats

trj: 3x700
0 481 3848
1 449 3592
2 439 3512
3 437 3496
4 440 3520
5 474 3792
6 461 3688
7 479 3832
8 440 3520
9 458 3664
10 443 3544
11 478 3824
12 470 3760
13 434 3472
14 447 3576
15 414 3312
16 456 3648
17 471 3768
18 473 3784
19 482 3856
20 453 3624
21 472 3776
22 463 3704
23 482 3856
24 463 3704
25 448 3584
26 435 3480
27 446 3568
28 466 3728
29 486 3888
30 451 3608
31 440 3520
32 455 3640
33 444 3552
34 460 3680
35 453 3624
36 438 3504
37 434 3472
38 443 3544
39 480 3840
40 470 3760
41 464 3712
42 459 3672
43 439 3512
44 454 3632
45 436 3488
46 428 3424
47 437 3496
48 437 3496
49 449 3592
50 452 3616
51 446 3568
52 432 3456
53 450 3600
54 427 3416
55 428 3424
56 486 3888
57 462 3696
58 463 3704
59 430 3440
60 464 3712
61 452 3616
62 454 3632
63 454 3632
64 447 3576
65 427 3416
66 440 3520
67 455 3640
68 467 3736
69 460 3680
70 464 3712
71 441 3528
72 451 3608
73 437 3496
74 426 3408
75 424 3392
76 430 3440
77 414 3312
78 446 3568
79 446 3568
80 438 3504
81 505 4040
82 407 3256
83 

810 478 3824
811 440 3520
812 473 3784
813 461 3688
814 457 3656
815 491 3928
816 473 3784
817 425 3400
818 479 3832
819 441 3528
820 462 3696
821 467 3736
822 444 3552
823 461 3688
824 423 3384
825 438 3504
826 454 3632
827 455 3640
828 474 3792
829 429 3432
830 447 3576
831 429 3432
832 439 3512
833 435 3480
834 467 3736
835 459 3672
836 429 3432
837 451 3608
838 468 3744
839 466 3728
840 452 3616
841 455 3640
842 442 3536
843 446 3568
844 458 3664
845 475 3800
846 471 3768
847 451 3608
848 451 3608
849 483 3864
850 424 3392
851 465 3720
852 428 3424
853 462 3696
854 466 3728
855 430 3440
856 457 3656
857 438 3504
858 453 3624
859 433 3464
860 468 3744
861 479 3832
862 459 3672
863 460 3680
864 452 3616
865 478 3824
866 433 3464
867 477 3816
868 457 3656
869 453 3624
870 463 3704
871 449 3592
872 450 3600
873 438 3504
874 457 3656
875 473 3784
876 461 3688
877 452 3616
878 460 3680
879 437 3496
880 431 3448
881 455 3640
882 486 3888
883 450 3600
884 450 3600
885 446 3568
886 447 3576

1477 451 3608
1478 456 3648
1479 452 3616
1480 436 3488
1481 439 3512
1482 453 3624
1483 442 3536
1484 461 3688
1485 434 3472
1486 472 3776
1487 451 3608
1488 463 3704
1489 466 3728
1490 464 3712
1491 482 3856
1492 442 3536
1493 428 3424
1494 472 3776
1495 466 3728
1496 463 3704
1497 444 3552
1498 466 3728
1499 481 3848
1500 444 3552
1501 454 3632
1502 434 3472
1503 433 3464
1504 491 3928
1505 447 3576
1506 451 3608
1507 448 3584
1508 473 3784
1509 449 3592
1510 446 3568
1511 433 3464
1512 428 3424
1513 470 3760
1514 446 3568
1515 467 3736
1516 452 3616
1517 435 3480
1518 445 3560
1519 452 3616
1520 442 3536
1521 461 3688
1522 425 3400
1523 432 3456
1524 465 3720
1525 447 3576
1526 437 3496
1527 471 3768
1528 447 3576
1529 471 3768
1530 448 3584
1531 442 3536
1532 477 3816
1533 479 3832
1534 446 3568
1535 473 3784
1536 452 3616
1537 410 3280
1538 441 3528
1539 451 3608
1540 458 3664
1541 434 3472
1542 414 3312
1543 460 3680
1544 504 4032
1545 460 3680
1546 484 3872
1547 487 3896
1548 4

2063 479 3832
2064 466 3728
2065 461 3688
2066 440 3520
2067 444 3552
2068 445 3560
2069 435 3480
2070 450 3600
2071 433 3464
2072 466 3728
2073 482 3856
2074 445 3560
2075 442 3536
2076 452 3616
2077 434 3472
2078 467 3736
2079 468 3744
2080 444 3552
2081 469 3752
2082 433 3464
2083 439 3512
2084 474 3792
2085 447 3576
2086 475 3800
2087 469 3752
2088 440 3520
2089 454 3632
2090 472 3776
2091 451 3608
2092 460 3680
2093 458 3664
2094 445 3560
2095 468 3744
2096 453 3624
2097 457 3656
2098 485 3880
2099 428 3424
2100 428 3424
2101 437 3496
2102 454 3632
2103 453 3624
2104 463 3704
2105 460 3680
2106 462 3696
2107 441 3528
2108 447 3576
2109 466 3728
2110 455 3640
2111 446 3568
2112 446 3568
2113 493 3944
2114 436 3488
2115 449 3592
2116 448 3584
2117 452 3616
2118 449 3592
2119 439 3512
2120 457 3656
2121 427 3416
2122 462 3696
2123 449 3592
2124 460 3680
2125 447 3576
2126 477 3816
2127 446 3568
2128 456 3648
2129 455 3640
2130 453 3624
2131 456 3648
2132 426 3408
2133 461 3688
2134 4

2649 461 3688
2650 436 3488
2651 457 3656
2652 461 3688
2653 466 3728
2654 450 3600
2655 473 3784
2656 439 3512
2657 468 3744
2658 444 3552
2659 450 3600
2660 435 3480
2661 452 3616
2662 469 3752
2663 452 3616
2664 446 3568
2665 485 3880
2666 444 3552
2667 469 3752
2668 456 3648
2669 454 3632
2670 448 3584
2671 486 3888
2672 466 3728
2673 457 3656
2674 460 3680
2675 460 3680
2676 443 3544
2677 442 3536
2678 455 3640
2679 419 3352
2680 428 3424
2681 442 3536
2682 457 3656
2683 453 3624
2684 460 3680
2685 445 3560
2686 476 3808
2687 456 3648
2688 454 3632
2689 490 3920
2690 475 3800
2691 428 3424
2692 469 3752
2693 456 3648
2694 470 3760
2695 454 3632
2696 452 3616
2697 477 3816
2698 469 3752
2699 451 3608
2700 453 3624
2701 446 3568
2702 432 3456
2703 458 3664
2704 444 3552
2705 455 3640
2706 428 3424
2707 436 3488
2708 449 3592
2709 429 3432
2710 476 3808
2711 479 3832
2712 466 3728
2713 458 3664
2714 474 3792
2715 454 3632
2716 458 3664
2717 462 3696
2718 471 3768
2719 421 3368
2720 4

...

9681 471 3768
9682 473 3784
9683 427 3416
9684 433 3464
9685 487 3896
9686 454 3632
9687 468 3744
9688 443 3544
9689 427 3416
9690 461 3688
9691 439 3512
9692 447 3576
9693 461 3688
9694 447 3576
9695 470 3760
9696 466 3728
9697 463 3704
9698 452 3616
9699 450 3600
9700 456 3648
9701 462 3696
9702 432 3456
9703 439 3512
9704 454 3632
9705 427 3416
9706 459 3672
9707 438 3504
9708 422 3376
9709 432 3456
9710 437 3496
9711 445 3560
9712 429 3432
9713 457 3656
9714 429 3432
9715 467 3736
9716 427 3416
9717 453 3624
9718 440 3520
9719 437 3496
9720 481 3848
9721 464 3712
9722 447 3576
9723 461 3688
9724 452 3616
9725 453 3624
9726 470 3760
9727 443 3544
9728 456 3648
9729 460 3680
9730 445 3560
9731 506 4048
9732 462 3696
9733 434 3472
9734 429 3432
9735 451 3608
9736 476 3808
9737 448 3584
9738 436 3488
9739 434 3472
9740 455 3640
9741 463 3704
9742 433 3464
9743 475 3800
9744 441 3528
9745 482 3856
9746 471 3768
9747 417 3336
9748 482 3856
9749 462 3696
9750 444 3552
9751 458 3664
9752 4

10249 440 3520
10250 454 3632
10251 473 3784
10252 451 3608
10253 446 3568
10254 459 3672
10255 418 3344
10256 461 3688
10257 461 3688
10258 454 3632
10259 463 3704
10260 476 3808
10261 443 3544
10262 457 3656
10263 464 3712
10264 458 3664
10265 465 3720
10266 463 3704
10267 440 3520
10268 465 3720
10269 486 3888
10270 433 3464
10271 447 3576
10272 461 3688
10273 440 3520
10274 453 3624
10275 436 3488
10276 468 3744
10277 453 3624
10278 451 3608
10279 476 3808
10280 474 3792
10281 472 3776
10282 451 3608
10283 449 3592
10284 462 3696
10285 467 3736
10286 435 3480
10287 480 3840
10288 483 3864
10289 458 3664
10290 429 3432
10291 468 3744
10292 460 3680
10293 462 3696
10294 467 3736
10295 455 3640
10296 474 3792
10297 468 3744
10298 488 3904
10299 461 3688
10300 400 3200
10301 451 3608
10302 428 3424
10303 422 3376
10304 452 3616
10305 469 3752
10306 466 3728
10307 459 3672
10308 439 3512
10309 445 3560
10310 454 3632
10311 430 3440
10312 449 3592
10313 459 3672
10314 445 3560
10315 427 

10796 495 3960
10797 464 3712
10798 444 3552
10799 436 3488
10800 448 3584
10801 448 3584
10802 444 3552
10803 435 3480
10804 468 3744
10805 510 4080
10806 447 3576
10807 447 3576
10808 484 3872
10809 447 3576
10810 459 3672
10811 470 3760
10812 449 3592
10813 468 3744
10814 450 3600
10815 451 3608
10816 461 3688
10817 442 3536
10818 431 3448
10819 430 3440
10820 414 3312
10821 447 3576
10822 446 3568
10823 443 3544
10824 463 3704
10825 470 3760
10826 412 3296
10827 426 3408
10828 454 3632
10829 464 3712
10830 453 3624
10831 439 3512
10832 459 3672
10833 443 3544
10834 458 3664
10835 468 3744
10836 493 3944
10837 457 3656
10838 437 3496
10839 440 3520
10840 482 3856
10841 453 3624
10842 441 3528
10843 444 3552
10844 476 3808
10845 461 3688
10846 494 3952
10847 464 3712
10848 434 3472
10849 451 3608
10850 436 3488
10851 439 3512
10852 434 3472
10853 460 3680
10854 421 3368
10855 444 3552
10856 466 3728
10857 455 3640
10858 411 3288
10859 433 3464
10860 441 3528
10861 454 3632
10862 447 

11343 464 3712
11344 484 3872
11345 455 3640
11346 479 3832
11347 448 3584
11348 466 3728
11349 416 3328
11350 457 3656
11351 440 3520
11352 462 3696
11353 454 3632
11354 437 3496
11355 472 3776
11356 422 3376
11357 426 3408
11358 461 3688
11359 428 3424
11360 449 3592
11361 434 3472
11362 395 3160
11363 457 3656
11364 454 3632
11365 466 3728
11366 433 3464
11367 478 3824
11368 445 3560
11369 476 3808
11370 424 3392
11371 448 3584
11372 462 3696
11373 443 3544
11374 468 3744
11375 452 3616
11376 435 3480
11377 448 3584
11378 456 3648
11379 459 3672
11380 465 3720
11381 438 3504
11382 456 3648
11383 458 3664
11384 461 3688
11385 445 3560
11386 453 3624
11387 457 3656
11388 450 3600
11389 457 3656
11390 440 3520
11391 460 3680
11392 460 3680
11393 457 3656
11394 464 3712
11395 460 3680
11396 455 3640
11397 453 3624
11398 446 3568
11399 446 3568
11400 454 3632
11401 472 3776
11402 450 3600
11403 427 3416
11404 455 3640
11405 473 3784
11406 429 3432
11407 444 3552
11408 459 3672
11409 459 

11890 476 3808
11891 444 3552
11892 427 3416
11893 457 3656
11894 455 3640
11895 482 3856
11896 483 3864
11897 460 3680
11898 462 3696
11899 461 3688
11900 432 3456
11901 443 3544
11902 461 3688
11903 444 3552
11904 424 3392
11905 462 3696
11906 448 3584
11907 462 3696
11908 443 3544
11909 445 3560
11910 454 3632
11911 470 3760
11912 454 3632
11913 435 3480
11914 461 3688
11915 428 3424
11916 435 3480
11917 442 3536
11918 455 3640
11919 438 3504
11920 468 3744
11921 420 3360
11922 448 3584
11923 471 3768
11924 468 3744
11925 450 3600
11926 470 3760
11927 422 3376
11928 463 3704
11929 478 3824
11930 453 3624
11931 448 3584
11932 443 3544
11933 440 3520
11934 444 3552
11935 439 3512
11936 434 3472
11937 471 3768
11938 447 3576
11939 465 3720
11940 459 3672
11941 444 3552
11942 465 3720
11943 418 3344
11944 406 3248
11945 468 3744
11946 452 3616
11947 401 3208
11948 446 3568
11949 465 3720
11950 453 3624
11951 404 3232
11952 464 3712
11953 447 3576
11954 437 3496
11955 455 3640
11956 425 

In [305]:
ti[ind[2]], ti_core[2], np.array_equal(dist[:,0], np.zeros(dist.shape[0]))

(array([5, 1, 5, 4, 5, 5, 4, 4, 2, 4, 4, 4, 2, 5, 3, 5, 2, 6, 5, 2, 5, 3,
        3, 5, 2, 3, 2, 6, 2, 4, 4, 2, 3, 2, 5, 3, 2, 3, 3, 6, 5, 3, 3, 4,
        5, 6, 2, 2, 1, 4, 5, 2, 3, 5, 5, 5, 2, 5, 5, 2, 6, 3, 2, 4, 2, 4,
        4, 4, 5, 4, 2, 4, 4, 2, 3, 4, 5, 2, 2, 5, 5, 3, 2, 4, 5, 2, 2, 4,
        2, 6, 5, 2, 2, 4, 3, 2, 5, 3, 3, 3, 4, 2, 3, 3, 6, 5, 6, 3, 5, 5,
        2, 2, 3, 3, 3, 1, 4, 2, 4, 1, 4, 5, 5, 4, 3, 5, 5, 2, 4, 6, 5, 5,
        5, 5, 2, 4, 4, 2, 6, 6, 6, 6, 2, 2, 4, 5, 2, 3, 2, 3, 2, 5, 2, 3,
        3, 5, 2, 5, 2, 3, 3, 5, 3, 3, 2, 3, 3, 2, 2, 5, 2, 3, 4, 2, 4, 5,
        5, 4, 6, 4, 5, 4, 4, 4, 3, 3, 5, 5, 5, 2, 6, 5, 4, 1, 4, 2, 5, 3,
        4, 2, 4]), 5, True)

In [306]:
knn_stats[-1][0]

array([[[ 10,   0],
        [  8,   2],
        [  9,   1],
        [  9,   1],
        [  5,   5],
        [  9,   1]],

       [[100,   1],
        [ 78,  23],
        [ 81,  20],
        [ 79,  22],
        [ 75,  26],
        [ 92,   9]],

       [[ 84,   0],
        [ 64,  20],
        [ 65,  19],
        [ 65,  19],
        [ 67,  17],
        [ 75,   9]],

       [[ 86,   1],
        [ 67,  20],
        [ 71,  16],
        [ 66,  21],
        [ 63,  24],
        [ 82,   5]],

       [[ 98,   4],
        [ 76,  26],
        [ 83,  19],
        [ 81,  21],
        [ 78,  24],
        [ 94,   8]],

       [[ 36,   1],
        [ 30,   7],
        [ 28,   9],
        [ 30,   7],
        [ 31,   6],
        [ 30,   7]]])

In [307]:
trjs['tx700']['atom_type_core'][1]

array([5, 5, 4, 2, 4, 2, 3, 5, 4, 5, 5, 5, 4, 4, 5, 6, 3, 5, 4, 4, 5, 4,
       6, 4, 5, 3, 4, 1, 2, 4, 3, 2, 3, 6, 3, 3, 5, 4, 1, 4, 5, 5, 2, 2,
       3, 4, 3, 2, 2, 6, 5, 4, 2, 3, 6, 4, 2, 5, 5, 2, 2, 2, 2, 5, 5, 4,
       3, 4, 2, 3, 5, 4, 2, 5, 5, 5, 2, 6, 6, 6, 4, 1, 6, 3, 5, 4, 4, 4,
       6, 3, 3, 2, 6, 5, 2, 2, 4, 3, 4, 2, 6, 3, 5, 2, 2, 5, 3, 4, 3, 4,
       3, 2, 6, 5, 2, 5, 5, 2, 6, 6, 4, 2, 3, 5, 3, 4, 3, 4, 2, 6, 4, 2,
       3, 5, 4, 5, 5, 4, 2, 4, 3, 5, 4, 6, 3, 3, 5, 5, 5, 3, 3, 6, 2, 4,
       4, 3, 5, 2, 5, 5, 5, 3, 5, 5, 5, 3, 5, 3, 3, 3, 5, 5, 6, 5, 5, 5,
       4, 2, 3, 5, 2, 2, 2, 5, 2, 3, 4, 5, 4, 3, 3, 5, 2, 2, 5, 5, 5, 3,
       2, 4, 2, 5, 2, 4, 4, 5, 4, 6, 2, 5, 5, 2, 3, 3, 5, 3, 6, 2, 6, 2,
       4, 4, 4, 3, 3, 3, 6, 2, 4, 4, 3, 2, 2, 6, 6, 2, 4, 3, 5, 5, 3, 2,
       2, 3, 4, 5, 3, 5, 4, 5, 2, 3, 3, 1, 2, 2, 2, 3, 6, 4, 2, 2, 2, 4,
       5, 5, 4, 3, 4, 2, 3, 4, 2, 4, 2, 5, 4, 5, 3, 2, 4, 1, 6, 6, 3, 3,
       6, 5, 3, 4, 3, 2, 2, 2, 6, 4, 3, 3, 3, 3, 4,

In [308]:
# with open('../data/working/trjs_knn.pickle', 'wb') as fo:
#     pickle.dump(trjs, fo)

In [309]:
trjs['3x700'].keys()

dict_keys(['energy', 'interaction_stats', 'temp', 'box_latt', 'xyz_latt', 'atom_type', 'ref_params', 'box', 'xyz', 'xyz_noise', 'atom_type_noise', 'box_noise', 'xyz_core', 'atom_type_core', 'knn'])

In [310]:
knn_all = {}
ppp_all = {}
conc_all = []

for key in ['3x700']:
    hists = trjs[key]['knn']
    print(len(hists), len(hists[0]), hists[0][0].shape)
    
    # sum histograms
    print(key)
    knn_trj = []
    ppp_trj = []
    for i, k in enumerate(knns):
        hhh = np.zeros((6, 6, k+1), dtype=float)
        for hst in hists:
            hhh += hst[i]
        knn_trj.append(hhh)
        hsum = np.sum(hhh, axis=2)
        #print(i, hsum[:,0])
        if i == 0:
            counts_all = hsum[:,1]
            conc_all = counts_all/np.sum(counts_all)
        ppp = hhh/hsum[:,:,None]
        ppp_trj.append(ppp)
    
    knn_all[key] = knn_trj
    ppp_all[key] = ppp_trj
    # get probability distributions
tot_counts_all = np.sum(counts_all)
print(conc_all, counts_all/10001, tot_counts_all)
print(conc, counts, tot_counts)

10001 16 (6, 6, 2)
3x700
[0.02316372 0.22774546 0.22792602 0.21245635 0.21762418 0.09108426] [ 10.48035196 103.04269573 103.12438756  96.12518748  98.46335366
  41.21077892] 4524920.0
[0.02325016 0.22784748 0.22782936 0.21274711 0.21713616 0.09118974] [ 30788 301717 301693 281721 287533 120754] 1324206


In [311]:
trjs['tx700']['knn'][0][0].shape

(6, 6, 2)

In [312]:
#  optimize only unlike parameters

select = []
for i in range(1, ntype):
    for j in range(i, ntype):
        if i == 1 and (j == 3 or j == 5):
            select.append(True)
        else:
            select.append(False)
#         #print(i,j)
#         if i == j:
#             select.append(False)
#         else:
#             select.append(True)

#         hist_ave = average_histogram(pars_in, params_list, X_list[key], df_est[key][0], hist_list[key])
#         cb1 = np.sqrt(hist_ave).dot(np.sqrt(hist_targ[key]))
#         s2 = np.arccos(cb1)**2

In [313]:
sum(select)

2

In [314]:
X_list, beta_ref, pars_list, ene_ref, hist_list = data_lists(trjs, minval=-10000)

In [315]:
len(trjs['tx700']['temp']), len(beta_ref[0]), len(hist_list[0])

(12001, 10000, 10000)

In [316]:
pars_list

[array([ 0.  , -2.97, -5.44, -1.11, -5.06,  0.  , -0.78, -0.06, -0.48,
         0.  , -0.24, -0.17,  0.  ,  0.04,  0.  ]),
 array([ 0.  , -2.97, -7.  , -1.05, -4.  ,  0.  , -0.78, -0.06, -0.48,
         0.  , -0.24, -0.17,  0.  ,  0.04,  0.  ])]

In [355]:
X_ref, pars_ref, hist_ref = make_reference_matrices(hist_list, X_list, pars_list, select, knn_max=16)

In [356]:
targets[1].shape

(6, 6, 2)

In [357]:
hist_targ = make_target_matrices(targets)[:10,:,:]

In [358]:
X_ref[1].shape, pars_ref[1].shape, beta_ref[1].shape, hist_ref[0].shape, hist_targ.shape

((10000, 2), (2,), (10000,), (10000, 16, 6, 6), (10, 6, 6))

In [359]:
pars_in = pars_ref[1]

In [360]:
pars_in

array([-7., -4.])

In [237]:
output = fmin(loss_sd2_hist, pars_in, args=(X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ))

[0.02208874 0.22694179 0.23120937 0.2117249  0.21739916 0.09063605]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999546349665899
0.0038170811732190094 [-6. -5.]
[0.02208746 0.22694315 0.23121662 0.21171802 0.21740026 0.09063449]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999547117546566
0.003810643116583101 [-6.3 -5. ]
[0.02209284 0.22693924 0.2312063  0.21172066 0.21740514 0.09063581]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999546545806492
0.0038154166041221146 [-6.   -5.25]
[0.02209155 0.22694062 0.23121354 0.21171379 0.21740625 0.09063426]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999547312433941
0.003808989182991927 [-6.3  -5.25]
[0.02209296 0.22694002 0.23121562 0.21170824 0.2174098  0.09063336]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.999954779262266
0.003804953181311746 [-6.45  -5.375]
[0.02208757 0.22694384 0.23122599 0.21170565 0.217404

[0.02212958 0.22691936 0.23122123 0.21162712 0.21747991 0.0906228 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999553707985956
0.0037551425581271226 [-8. -8.]
[0.02212958 0.22691936 0.23122123 0.21162712 0.21747991 0.0906228 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999553707985956
0.0037551425581271226 [-8. -8.]
[0.02212958 0.22691936 0.23122123 0.21162712 0.21747991 0.0906228 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999553707985956
0.0037551425581271226 [-8. -8.]
[0.02212958 0.22691936 0.23122123 0.21162712 0.21747991 0.0906228 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999553707985956
0.0037551425581271226 [-8. -8.]
[0.02212958 0.22691936 0.23122123 0.21162712 0.21747991 0.0906228 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999553707985956
0.0037551425581271226 [-8. -8.]
[0.02212958 0.22691936 0.23122123 0.21162712 0.21747991 0.0906228

In [342]:
pars_in = pars_ref[0]
output = fmin(loss_sd2_hist, pars_in, args=(X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ))

[0.02217415 0.22772579 0.23067028 0.20984925 0.21857788 0.09100265]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999494781124738
[0.02190257 0.22855006 0.23098473 0.2087677  0.2192735  0.09052143]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999652265738994
[0.02192957 0.22885657 0.2298603  0.20931562 0.21942597 0.09061196]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999656400159949
[0.0218049  0.22840474 0.2295792  0.20967534 0.21920442 0.0913314 ]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999700941501011
[0.02198929 0.2283185  0.22937813 0.20968154 0.21932877 0.09130377]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999634047547733
[0.02195022 0.22826577 0.22925521 0.20999854 0.21934815 0.09118211]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999680410240857
[0.0219509  0.22823355 0.22920416 0.21027806 0.21917277 0.09116056]
[0

[0.02208611 0.22785799 0.22891239 0.21091191 0.21919721 0.09103438]
[0.02323633 0.2270495  0.23325971 0.21066    0.21568468 0.09010978]
0.999973808418559
0.13579296089228238 [-8.      -2.08725]
[0.02213495 0.22771581 0.23070543 0.20992727 0.21847736 0.09103918]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999492305790411
[0.02189185 0.22851822 0.23101472 0.20882476 0.21920673 0.09054372]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999657857118613
[0.02191057 0.22883855 0.22991156 0.20930128 0.21941469 0.09062335]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.99996589092864
[0.02178151 0.22834083 0.22963805 0.20971524 0.21918079 0.09134358]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999705499439558
[0.02198045 0.22826404 0.22942133 0.20969536 0.21929845 0.09134037]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999637354556729
[0.02194483 0.22823903 0.2293115 

[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0.02294725 0.2262042  0.23400351 0.21066243 0.21563596 0.09054664]
0.9999726980952441
[0.02201316 0.22800818 0.22904358 0.21073538 0.21914232 0.09105738]
[0.02307176 0.22660344 0.23365164 0.21060514 0.21566122 0.0904068 ]
0.9999736657155279
[0.02208465 0.22784112 0.22893088 0.210922   0.21918007 0.09104128]
[0.02323633 0.2270495  0.23325971 0.21066    0.21568468 0.09010978]
0.9999739397868801
0.13485252066070746 [-8.  1.]
[0.02211458 0.22770379 0.23073639 0.2099

[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0.02294725 0.2262042  0.23400351 0.21066243 0.21563596 0.09054664]
0.9999726980952441
[0.02201316 0.22800818 0.22904358 0.21073538 0.21914232 0.09105738]
[0

[0.02188371 0.22849828 0.23104032 0.2088437  0.21919239 0.0905416 ]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999660436490166
[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0

[0.02188371 0.22849828 0.23104032 0.2088437  0.21919239 0.0905416 ]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999660436490166
[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0

[0.02201316 0.22800818 0.22904358 0.21073538 0.21914232 0.09105738]
[0.02307176 0.22660344 0.23365164 0.21060514 0.21566122 0.0904068 ]
0.9999736657155279
[0.02208465 0.22784112 0.22893088 0.210922   0.21918007 0.09104128]
[0.02323633 0.2270495  0.23325971 0.21066    0.21568468 0.09010978]
0.9999739397868801
0.13485252066070746 [-8.  1.]
[0.02211458 0.22770379 0.23073639 0.20994903 0.21844492 0.09105129]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999492446036857
[0.02188371 0.22849828 0.23104032 0.2088437  0.21919239 0.0905416 ]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999660436490166
[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.2097

[0.02188371 0.22849828 0.23104032 0.2088437  0.21919239 0.0905416 ]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999660436490166
[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0

In [361]:
pars_ref[:], pars_list

([array([-5.44, -5.06]), array([-7., -4.])],
 [array([ 0.  , -2.97, -5.44, -1.11, -5.06,  0.  , -0.78, -0.06, -0.48,
          0.  , -0.24, -0.17,  0.  ,  0.04,  0.  ]),
  array([ 0.  , -2.97, -7.  , -1.05, -4.  ,  0.  , -0.78, -0.06, -0.48,
          0.  , -0.24, -0.17,  0.  ,  0.04,  0.  ])])

In [362]:
loss_sd2_hist(np.array([-5.44, -5.06]), X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ)

0
[0.02217415 0.22772579 0.23067028 0.20984925 0.21857788 0.09100265]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999494781124738
1
[0.02329492 0.22705077 0.22802042 0.21291712 0.21757921 0.09113758]
[0.02334638 0.2283862  0.22862152 0.21506246 0.21700799 0.08757544]
0.9999781808303895
2
[0.0235626  0.22837096 0.22623684 0.21267452 0.21806123 0.09109384]
[0.02424319 0.22799004 0.22757571 0.21341231 0.21787711 0.08890163]
0.9999894966391973
3
[0.02273943 0.22782826 0.22832975 0.2128583  0.21732499 0.09091928]
[0.02299438 0.22970954 0.22732065 0.21343812 0.21862765 0.08790967]
0.9999833183191307
4
[0.0232005  0.2276179  0.22812949 0.21273592 0.21750473 0.09081146]
[0.02346861 0.22945888 0.229518   0.21476839 0.21448321 0.08830291]
0.9999802250176928
5
[0.02305648 0.22760919 0.22813931 0.21223337 0.21809371 0.09086794]
[0.02312967 0.22453915 0.22450602 0.20746311 0.21409643 0.10626563]
0.9996635403132106
0
[0.02190257 0.22855006 0.23098473 0.2087677  0.2192735  

6.574445524761564

In [363]:
X_ref[0][-1], X_ref[1][-2]

(array([141, 105]), array([161, 104]))

In [364]:
# loss_sd2_hist(np.array([-7., -4.]), X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ)

0
[0.02216507 0.2277279  0.23067004 0.2098804  0.21854046 0.09101612]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999493186530525
1
[0.02329583 0.22703052 0.22803246 0.21292995 0.21757022 0.09114101]
[0.02334638 0.2283862  0.22862152 0.21506246 0.21700799 0.08757544]
0.9999781632938969
2
[0.02356245 0.22836014 0.22625112 0.21267451 0.21804447 0.09110732]
[0.02424319 0.22799004 0.22757571 0.21341231 0.21787711 0.08890163]
0.999989442502041
3
[0.02273402 0.22782947 0.22832582 0.21287246 0.21733    0.09090823]
[0.02299438 0.22970954 0.22732065 0.21343812 0.21862765 0.08790967]
0.9999834189373424
4
[0.0232009  0.22761029 0.22813811 0.21273997 0.21751972 0.090791  ]
[0.02346861 0.22945888 0.229518   0.21476839 0.21448321 0.08830291]
0.9999803228383659
5
[0.02305554 0.22757641 0.22819622 0.21223844 0.21806621 0.09086718]
[0.02312967 0.22453915 0.22450602 0.20746311 0.21409643 0.10626563]
0.9996634873642769
0
[0.02190213 0.22854497 0.23098382 0.20878826 0.21924384 0

6.57448816626807

In [365]:
loss_sd2_hist(np.array([-7., -4.]), X_ref[1], pars_ref[1], beta_ref[1], hist_ref[1], hist_targ)

0
[0.02151574 0.23008005 0.22879197 0.21111185 0.21813428 0.0903661 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999265680773258
1
[0.02331377 0.22738764 0.22772634 0.2125209  0.21842431 0.09062703]
[0.02334638 0.2283862  0.22862152 0.21506246 0.21700799 0.08757544]
0.9999810148008674
2
[0.02327712 0.22791219 0.22726153 0.21222279 0.21817274 0.09115363]
[0.02424319 0.22799004 0.22757571 0.21341231 0.21787711 0.08890163]
0.9999871089987679
3
[0.02272127 0.22805116 0.22797522 0.21270252 0.21742238 0.09112746]
[0.02299438 0.22970954 0.22732065 0.21343812 0.21862765 0.08790967]
0.9999822452077978
4
[0.02306455 0.22742091 0.22858582 0.21240103 0.21736229 0.09116539]
[0.02346861 0.22945888 0.229518   0.21476839 0.21448321 0.08830291]
0.9999768824814581
5
[0.02309589 0.22739428 0.22752774 0.21287138 0.21797684 0.09113388]
[0.02312967 0.22453915 0.22450602 0.20746311 0.21409643 0.10626563]
0.9996739144847849
0
[0.02217886 0.22880628 0.22997033 0.20942303 0.21826309 

6.716010197274094

In [255]:
loss_sd2_hist(np.array([-7.0, -4.0]), X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ)

[0.02206824 0.22695578 0.2312463  0.2117187  0.21737919 0.0906318 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.999954813962649
0.0038021500690616243 [-7. -4.]


0.0038021500690616243

## Compare distributions, statistical testing

In [27]:
for i, k in enumerate(targets.keys()):
    if k > 200:
        continue
    print('null')
    get_s2_two(k, targets[k], null_counts[k])
#     print('txinf')
#     get_s2_two(k, targets[k], knn_all['txinf'][i])
#     print('tx700')
#     get_s2_two(k, targets[k], knn_all['tx700'][i])
#     print('tx500')
#     get_s2_two(k, targets[k], knn_all['tx500'][i])
    print('1x700')
    get_s2_two(k, targets[k], knn_all['1x700'][i])

null
0 [ 699 6982 7314 6435 6675 2683] [ 715.82589416 7014.96821189 7014.41020808 6550.05803327 6685.18795716
 2807.54969544] [ -16.82589416  -32.96821189  299.58979192 -115.05803327  -10.18795716
 -124.54969544]
1 0 p-value 0.06477022670197327
1 [ 7044 68908 68979 64888 65475 26423] [ 7014.96821189 68745.45810017 68739.98976066 64189.41989162
 65513.66944494 27513.49459072] [   29.03178811   162.54189983   239.01023934   698.58010838
   -38.66944494 -1090.49459072]
1 1 p-value 7.107498401190488e-05
2 [ 7314 68783 68658 64385 65732 26821] [ 7014.41020808 68739.98976066 68734.52185612 64184.31396097
 65508.4581772  27511.30603698] [ 299.58979192   43.01023934  -76.52185612  200.68603903  223.5418228
 -690.30603698]
1 2 p-value 0.007483190721920082
3 [ 6478 64714 64041 60130 61592 24766] [ 6550.05803327 64189.41989162 64184.31396097 59935.32867318
 61171.8148785  25690.06456246] [ -72.05803327  524.58010838 -143.31396097  194.67132682  420.1851215
 -924.06456246]
1 3 p-value 0.0006887766

0 [ 4874 48802 50489 45386 46404 19561] [ 4854.24251119 49036.24159302 49463.50916255 45295.07379777
 47152.55403803 19714.37889743] [  19.75748881 -234.24159302 1025.49083745   90.92620223 -748.55403803
 -153.37889743]
7 0 p-value 4.547473927550435e-05
1 [ 48353 480116 482301 452122 460587 188540] [ 48802.86746767 479733.07288987 481592.4218316  449892.29081739
 459943.18114301 192055.16585045] [ -449.86746767   382.92711013   708.5781684   2229.70918261
   643.81885699 -3515.16585045]
7 1 p-value 2.045806987248768e-12
2 [ 50254 482122 480433 449732 460025 189285] [ 49216.07160707 481930.18379487 479879.16050305 448696.46534748
 459924.99839741 192204.12035012] [ 1037.92839293   191.81620513   553.83949695  1035.53465252
   100.00160259 -2919.12035012]
7 2 p-value 2.37741912295997e-10
3 [ 45051 451254 449022 421481 429955 175284] [ 45128.73684785 450348.5590962  449225.85317862 419521.29181232
 428408.64825423 179413.91081079] [  -77.73684785   905.4409038   -203.85317862  1959.708187

40 3 p-value 5.902804369896714e-41
4 [ 266366 2628203 2621686 2456121 2495756 1033188] [ 267407.51828643 2620546.77779741 2620338.32708808 2446872.59514003
 2497352.40858295 1048802.37310509] [ -1041.51828643   7656.22220259   1347.67291192   9248.40485997
  -1596.40858295 -15614.37310509]
40 4 p-value 2.6418330111670297e-30
5 [ 109971 1083446 1082970 1012553 1034562  506658] [ 112301.98781761 1100539.78362883 1100452.24147904 1027602.58249849
 1048802.37310509  440461.03147094] [ -2330.98781761 -17093.78362883 -17482.24147904 -15049.58249849
 -14240.37310509  66196.96852906]
40 5 p-value 0.0
1x700
0 [ 29110 279423 285460 260991 266525 110011] [ 27420.21282375 280704.19247102 281640.29419641 260054.30077662
 269358.7809021  112342.2188301 ] [ 1689.78717625 -1281.19247102  3819.70580359   936.69922338
 -2833.7809021  -2331.2188301 ]
40 0 p-value 3.0022190818624524e-38
1 [ 279459 2748873 2753290 2577326 2628427 1081305] [ 278877.20307188 2741901.88062306 2752754.56219961 2567412.75386286

In [ ]:
Al Fe Ni Cr Co
-16.82589416  -32.96821189  299.58979192 -115.05803327  -10.18795716 -124.54969544

In [28]:
 -16.82589416  -32.96821189  299.58979192 -115.05803327  -10.18795716 -124.54969544
20.47253984   11.75252954  194.89341128  -72.22808289   15.09589838 -169.98629614
18.87351129    3.42587269  232.80410678  -64.57453799 -103.62340862 -86.90554415


-24.30820551   -8.80891829  267.59700443 -145.58314801   10.44053525 -99.33726787

18.64444444    5.56239316  226.12991453  -82.77094017  -13.88888889 -153.67692308
18.87351129    3.42587269  232.80410678  -64.57453799 -103.62340862 -86.90554415
30.73242609   44.18882986  213.21814743 -100.79731438 -115.05498589 -72.28710312

-171.15009878  259.44646136 -294.4464846   140.7234631   169.98398605 -104.55732714

SyntaxError: invalid syntax (<ipython-input-28-99c728985025>, line 1)